In [46]:
import pandas as pd
from datetime import datetime, timezone
import numpy as np

In [47]:
# Load all CSV files
df_fact_player = pd.read_csv('../data/Fact_Player.csv')
df_gameweeks = pd.read_csv('../data/Gameweeks.csv')
df_player_history = pd.read_csv('../data/Player_history.csv')
df_players = pd.read_csv('../data/Players.csv')
df_positions = pd.read_csv('../data/Positions.csv')
df_teams = pd.read_csv('../data/Teams.csv')
df_fixtures=pd.read_csv('../data/Fixtures.csv')


In [48]:
# Create mappings of team IDs to team names and short names
team_name_mapping = pd.Series(df_teams.team_name.values, index=df_teams.id).to_dict()
team_short_name_mapping = pd.Series(df_teams.short_name.values, index=df_teams.id).to_dict()

# Replace team_a and team_h IDs with team names
df_fixtures['team_a'] = df_fixtures['team_a'].replace(team_name_mapping)
df_fixtures['team_h'] = df_fixtures['team_h'].replace(team_name_mapping)

# Add team_a_short and team_h_short columns using the original team IDs
df_fixtures['team_a_short'] = df_fixtures['team_a'].map(lambda x: team_short_name_mapping[df_teams[df_teams.team_name == x].id.values[0]] if x in team_name_mapping.values() else None)
df_fixtures['team_h_short'] = df_fixtures['team_h'].map(lambda x: team_short_name_mapping[df_teams[df_teams.team_name == x].id.values[0]] if x in team_name_mapping.values() else None)
df_fixtures = df_fixtures.drop(columns=['pulse_id'])
#filtered_fixtures = df_fixtures[(df_fixtures['finished'] == False) & (df_fixtures['finished_provisional'] == False)]

# Display the updated df_fixtures
#filtered_fixtures

df_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,team_a_short,team_h_short
0,2444470,1,True,True,1,2024-08-16T19:00:00Z,90,False,True,Fulham,0.0,Man Utd,1.0,2,3,FUL,MUN
1,2444473,1,True,True,4,2024-08-17T11:30:00Z,90,False,True,Liverpool,2.0,Ipswich,0.0,4,2,LIV,IPS
2,2444471,1,True,True,2,2024-08-17T14:00:00Z,90,False,True,Wolves,0.0,Arsenal,2.0,2,5,WOL,ARS
3,2444472,1,True,True,3,2024-08-17T14:00:00Z,90,False,True,Brighton,3.0,Everton,0.0,2,3,BHA,EVE
4,2444474,1,True,True,5,2024-08-17T14:00:00Z,90,False,True,Southampton,0.0,Newcastle,1.0,2,4,SOU,NEW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2444845,38,False,False,376,2025-05-25T15:00:00Z,0,False,False,Everton,NaN,Newcastle,NaN,2,4,EVE,NEW
376,2444846,38,False,False,377,2025-05-25T15:00:00Z,0,False,False,Chelsea,NaN,Nott'm Forest,NaN,3,2,CHE,NFO
377,2444847,38,False,False,378,2025-05-25T15:00:00Z,0,False,False,Arsenal,NaN,Southampton,NaN,5,2,ARS,SOU
378,2444848,38,False,False,379,2025-05-25T15:00:00Z,0,False,False,Brighton,NaN,Spurs,NaN,2,4,BHA,TOT


In [68]:
import pandas as pd

# Sample DataFrame: df_fixtures
# df_fixtures = pd.read_csv('your_file.csv')  # Ensure your DataFrame is already loaded.
upcoming_gameweeks = df_fixtures[df_fixtures['finished'] == False]

# Create an empty DataFrame to hold the home/away labels
teams = upcoming_gameweeks['team_a_short'].unique()  # Get unique team short names
unique_gameweeks = upcoming_gameweeks['event'].unique()  # Get unique gameweeks from upcoming games

# Format gameweek numbers with 'GW' prefix
formatted_gameweeks = [f'GW{gw}' for gw in unique_gameweeks]

# Create matrix based on unique gameweeks
fdr_matrix = pd.DataFrame(index=teams, columns=formatted_gameweeks)

# Dictionary to store original FDR values for color coding
fdr_values = {}

# Populate the FDR matrix with team names and home/away labels
for index, row in upcoming_gameweeks.iterrows():
    gameweek = f'GW{row["event"]}'  # Format gameweek with 'GW'
    team_a = row['team_a_short']
    team_h = row['team_h_short']
    fdr_a = row['team_a_difficulty']  # FDR for team_a
    fdr_h = row['team_h_difficulty']  # FDR for team_h

    # Assign team name with home/away indication
    fdr_matrix.at[team_a, gameweek] = f"{team_h} (A)"  # Team A is playing away
    fdr_matrix.at[team_h, gameweek] = f"{team_a} (H)"  # Team H is playing at home

    # Store FDR values for coloring later
    fdr_values[(team_a, gameweek)] = fdr_a
    fdr_values[(team_h, gameweek)] = fdr_h

# Convert the FDR table to a proper format (e.g., string to prevent confusion with numerical FDR)
fdr_matrix = fdr_matrix.astype(str)

# Define a function to color the DataFrame based on original FDR values
def color_fdr(team, gameweek):
    fdr_value = fdr_values.get((team, gameweek), None)
    
    if fdr_value is None:  # Handle NaN values
        return 'background-color: white;'  # Neutral color for NaN

    # Color coding based on FDR value
    if fdr_value == 1:  # Class for FDR 1
        return 'background-color: #257d5a;'  # Green
    elif fdr_value == 2:  # Class for FDR 2
        return 'background-color: #00ff86;'  # Light Green
    elif fdr_value == 3:  # Class for FDR 3
        return 'background-color: #ebebe4;'  # Yellow
    elif fdr_value == 4:  # Class for FDR 4
        return 'background-color: #ff005a;'  # Orange
    elif fdr_value == 5:  # Class for FDR 5
        return 'background-color: #861d46;'  # Red
    else:
        return ''  # No color for other values

# Create a styled DataFrame to visualize FDR values with color coding
styled_fdr_table = fdr_matrix.copy()  # Copy to apply styles

# Apply color to each cell based on FDR values while retaining team names
def apply_color(row):
    return [color_fdr(row.name, col) for col in row.index]

# Applying the style to the entire DataFrame
styled_fdr_table = styled_fdr_table.style.apply(apply_color, axis=1)

# To display the styled DataFrame
styled_fdr_table


,GW8,GW9,GW10,GW11,GW12,GW13,GW14,GW15,GW16,GW17,GW18,GW19,GW20,GW21,GW22,GW23,GW24,GW25,GW26,GW27,GW28,GW29,GW30,GW31,GW32,GW33,GW34,GW35,GW36,GW37,GW38
WHU,TOT (A),MUN (H),NFO (A),EVE (H),NEW (A),ARS (H),LEI (A),WOL (H),BOU (A),BHA (H),SOU (A),LIV (H),MCI (A),FUL (H),CRY (H),AVL (A),CHE (A),BRE (H),ARS (A),LEI (H),NEW (H),EVE (A),WOL (A),BOU (H),LIV (A),SOU (H),BHA (A),TOT (H),MUN (A),NFO (H),IPS (A)
AVL,FUL (A),BOU (H),TOT (A),LIV (A),CRY (H),CHE (A),BRE (H),SOU (H),NFO (A),MCI (H),NEW (A),BHA (H),LEI (H),EVE (A),ARS (A),WHU (H),WOL (A),IPS (H),CHE (H),CRY (A),BRE (A),LIV (H),BHA (A),NFO (H),SOU (A),NEW (H),MCI (A),FUL (H),BOU (A),TOT (H),MUN (A)
EVE,IPS (A),FUL (H),SOU (A),WHU (A),BRE (H),MUN (A),WOL (H),LIV (H),ARS (A),CHE (H),MCI (A),NFO (H),BOU (A),AVL (H),TOT (H),BHA (A),LEI (H),CRY (A),MUN (H),BRE (A),WOL (A),WHU (H),LIV (A),ARS (H),NFO (A),MCI (H),CHE (A),IPS (H),FUL (A),SOU (H),NEW (A)
BRE,MUN (A),IPS (H),FUL (A),BOU (H),EVE (A),LEI (H),AVL (A),NEW (H),CHE (A),NFO (H),BHA (A),ARS (H),SOU (A),MCI (H),LIV (H),CRY (A),TOT (H),WHU (A),LEI (A),EVE (H),AVL (H),BOU (A),NEW (A),CHE (H),ARS (A),BHA (H),NFO (A),MUN (H),IPS (A),FUL (H),WOL (A)
BHA,NEW (A),WOL (H),LIV (A),MCI (H),BOU (A),SOU (H),FUL (A),LEI (A),CRY (H),WHU (A),BRE (H),AVL (A),ARS (H),IPS (A),MUN (A),EVE (H),NFO (A),CHE (H),SOU (A),BOU (H),FUL (H),MCI (A),AVL (H),CRY (A),LEI (H),BRE (A),WHU (H),NEW (H),WOL (A),LIV (H),TOT (A)
LEI,SOU (A),NFO (H),IPS (A),MUN (A),CHE (H),BRE (A),WHU (H),BHA (H),NEW (A),WOL (H),LIV (A),MCI (H),AVL (A),CRY (H),FUL (H),TOT (A),EVE (A),ARS (H),BRE (H),WHU (A),CHE (A),MUN (H),MCI (A),NEW (H),BHA (A),LIV (H),WOL (A),SOU (H),NFO (A),IPS (H),BOU (A)
ARS,BOU (A),LIV (H),NEW (A),CHE (A),NFO (H),WHU (A),MUN (H),FUL (A),EVE (H),CRY (A),IPS (H),BRE (A),BHA (A),TOT (H),AVL (H),WOL (A),MCI (H),LEI (A),WHU (H),NFO (A),MUN (A),CHE (H),FUL (H),EVE (A),BRE (H),IPS (A),CRY (H),BOU (H),LIV (A),NEW (H),SOU (A)
MCI,WOL (A),SOU (H),BOU (A),BHA (A),TOT (H),LIV (A),NFO (H),CRY (A),MUN (H),AVL (A),EVE (H),LEI (A),WHU (H),BRE (A),IPS (A),CHE (H),ARS (A),NEW (H),LIV (H),TOT (A),NFO (A),BHA (H),LEI (H),MUN (A),CRY (H),EVE (A),AVL (H),WOL (H),SOU (A),BOU (H),FUL (A)
CHE,LIV (A),NEW (H),MUN (A),ARS (H),LEI (A),AVL (H),SOU (A),TOT (A),BRE (H),EVE (A),FUL (H),IPS (A),CRY (A),BOU (H),WOL (H),MCI (A),WHU (H),BHA (A),AVL (A),SOU (H),LEI (H),ARS (A),TOT (H),BRE (A),IPS (H),FUL (A),EVE (H),LIV (H),NEW (A),MUN (H),NFO (A)
CRY,NFO (A),TOT (H),WOL (A),FUL (H),AVL (A),NEW (H),IPS (A),MCI (H),BHA (A),ARS (H),BOU (A),SOU (H),CHE (H),LEI (A),WHU (A),BRE (H),MUN (A),EVE (H),FUL (A),AVL (H),IPS (H),NEW (A),SOU (A),BHA (H),MCI (A),BOU (H),ARS (A),NFO (H),TOT (A),WOL (H),LIV (A)
